## CODIGO DE RE-ENTRENAMIENTO

*NOTA: Para poder usar este código de entrenamiento hay que lanzarlo desde exactamente el mismo entorno en el que fue creado.*

*Se puede instalar ese entorno en la nueva máquina usando el environment.yml que creamos en el set up del proyecto*

*Copiar el riesgos.yml al directorio y en el terminal o anaconda prompt ejecutar:*

conda env create --file riesgos.yml --name riesgos

In [5]:
import numpy as np
import pandas as pd
import pickle

from janitor import clean_names

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


ruta_proyecto = 'C:/Users/alvar/Proyectos ML/03_RIESGOS'
nombre_fichero_datos = 'trabajo.csv'
ruta_completa = ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,index_col=1)

variables_finales = ['ingresos_verificados',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'antigüedad_empleo',
                     'rating',
                     'ingresos',
                     'dti',
                     'num_lineas_credito',
                     'porc_uso_revolving',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'num_hipotecas',
                     'porc_tarjetas_75p',
                     'num_cancelaciones_12meses',
                     'num_derogatorios',
                     'estado',
                     'imp_amortizado',
                     'imp_recuperado'
                  ]

df = df[variables_finales]

#Cambiar la estructura

df = clean_names(df)

num_desv_tip = 4
def atipicos_desv_tip(variable, num_desv_tip = 4):
    #sacamos los nulos por ahora
    variable = variable.dropna()
    #calculamos los límites
    media = np.mean(variable)
    sd = np.std(variable)
    umbral = sd * num_desv_tip
    lim_inf = media - umbral
    lim_sup = media + umbral
    #encontramos los índices de los que están fuera de los límites
    indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
    return(indices)
    
var_atipicos_dt = ['ingresos']

for variable in var_atipicos_dt:
    df.drop(atipicos_desv_tip(df[variable], num_desv_tip),axis = 0, inplace = True)

a_eliminar=df.loc[(df['dti']>100)|(df['dti']<0)].index.values
df=df.loc[~df.index.isin(a_eliminar)]


#Calidad datos, creacion variables
var_imputar_desconocido = df.select_dtypes(exclude = 'number').columns.to_list()
valor='Desconocido'
var_imputar_moda = df.select_dtypes(include='number').columns.to_list()

def calidad_datos(temp,var_imputar_desconocido,valor,var_imputar_moda):
    for variable in var_imputar_desconocido:
        temp[variable]=temp[variable].fillna(valor)
        
    for variable in var_imputar_moda:
            if pd.api.types.is_integer_dtype(temp[variable]):
                    
                temp[variable] = temp[variable].fillna(int(temp[variable].mode()[0]))
            else:
                temp[variable] = temp[variable].fillna(temp[variable].mode()[0])
    return temp

def creacion_variables_pd(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_pd']=np.where(temp['estado'].isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']),1,0)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

def creacion_variables_ead(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_ead']=temp.apply(lambda x: ((x['principal']-x['imp_amortizado'])
                                             /x['principal']) if ((x['principal']-x['imp_amortizado'])/x['principal'])>=0 else 0 ,axis=1)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

def creacion_variables_lgd(temp):
#hacer una copia, sino se sobre escribe df
    temp=df.copy() 
#corregir valores variables
    temp['finalidad']=temp.apply(lambda x:'other' if x.finalidad in['renewable_energy','educational','wedding'] else x.finalidad, axis=1)
    temp['vivienda']=temp['vivienda'].replace(['ANY','OTHER','NONE'],'MORTGAGE')
    temp['porc_uso_revolving']=temp.apply(lambda x: 100 if x.porc_uso_revolving>100 else x.porc_uso_revolving,axis=1)
    temp['num_cancelaciones_12meses']=np.where(temp['num_cancelaciones_12meses']>0,1,0)
    temp['num_derogatorios']=np.where(temp['num_derogatorios']>0,1,0)

#Crear target   
    temp['target_lgd'] = temp.apply(lambda x: 
                              1 - (x['imp_recuperado'] / (x['principal'] - x['imp_amortizado'])) 
                              if (x['principal'] - x['imp_amortizado']) != 0 
                              else np.nan, axis=1)
    temp['target_lgd'] = temp['target_lgd'].apply(lambda x: 0 if np.isnan(x) or np.isinf(x) or x<0 else x)

#Eliminamos variables que no usamos en este modelo y la que usamos para crear la target
    temp.drop(columns=['estado','imp_recuperado','imp_amortizado'],inplace=True)

#Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

x_pd, y_pd = creacion_variables_pd(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))

x_ead, y_ead = creacion_variables_ead(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))

x_lgd, y_lgd = creacion_variables_lgd(calidad_datos(df,var_imputar_desconocido,valor,var_imputar_moda))


#6.CARGA PIPES DE ENTRENAMIENTO
ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_pd.pickle'
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_ead.pickle'
ruta_pipe_entrenamiento_lgd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_lgd.pickle'

with open(ruta_pipe_entrenamiento_pd, mode='rb') as file:
   pipe_entrenamiento_pd = pickle.load(file)

with open(ruta_pipe_entrenamiento_ead, mode='rb') as file:
   pipe_entrenamiento_ead = pickle.load(file)

with open(ruta_pipe_entrenamiento_lgd, mode='rb') as file:
   pipe_entrenamiento_lgd = pickle.load(file)

#entrenar los pipe
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lgd = pipe_entrenamiento_lgd.fit(x_lgd,y_lgd)

scoring_pd = pipe_ejecucion_pd.predict_proba(x_pd)[:, 1]
ead = pipe_ejecucion_ead.predict(x_ead)
lgd = pipe_ejecucion_lgd.predict(x_lgd)

ead= np.where(ead < 0, 0, ead)
ead = np.where(ead > 1, 1, ead)

lgd = np.where(lgd < 0, 0, lgd)
lgd = np.where(lgd > 1, 1, lgd)

#pipe ejecucion
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_pd, file)

ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
   pickle.dump(pipe_ejecucion_ead, file)

ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_lgd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_lgd, file)




